# Function Calling & Custom Tools

Understanding differences and how to use them together

In [11]:
import pandas as pd
import numpy as np
import json, os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool

In [12]:
os.environ["OPENAI_API_KEY"] = ""

### Converting a custom python function to an OpenAI function calling tool

Requires specifying type of arguments, as well as description of function and each arguments. This will be passed to convert_to_openai_tool()

In [9]:
def create_matrix(r:int, c:int):
    """Create a matrix of rows(r) by columns(c)

    Args:
        r: First dimension of matrix
        c: Second dimension of matrix
    """

    m = np.random.randn(r, c)

    return m

convert_to_openai_tool(create_matrix)

{'type': 'function',
 'function': {'name': 'create_matrix',
  'description': 'Create a matrix of rows(r) by columns(c)',
  'parameters': {'type': 'object',
   'properties': {'r': {'type': 'integer',
     'description': 'First dimension of matrix'},
    'c': {'type': 'integer', 'description': 'Second dimension of matrix'}},
   'required': ['r', 'c']}}}

In [10]:

print(json.dumps(convert_to_openai_tool(create_matrix), indent=2))

{
  "type": "function",
  "function": {
    "name": "create_matrix",
    "description": "Create a matrix of rows(r) by columns(c)",
    "parameters": {
      "type": "object",
      "properties": {
        "r": {
          "type": "integer",
          "description": "First dimension of matrix"
        },
        "c": {
          "type": "integer",
          "description": "Second dimension of matrix"
        }
      },
      "required": [
        "r",
        "c"
      ]
    }
  }
}


### Testing ("Binding function) custom function call built in an LLM

We'll incorporate the function call as part of our model so it is "bound" (**binding function**) and ready to be utilized

In [13]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

Test it without function calling. LLM will produce actual answer

In [17]:
llm_result = llm.invoke("can you create a random 3x5 matrix?")
llm_result

AIMessage(content='Sure, here is a random 3x5 matrix:\n\n[[4, 7, 2, 9, 1],\n [3, 8, 5, 6, 2],\n [1, 9, 4, 7, 3]]')

Notice that answer will be produced, but it will be produced as part of text

In [19]:
llm_result.content

'Sure, here is a random 3x5 matrix:\n\n[[4, 7, 2, 9, 1],\n [3, 8, 5, 6, 2],\n [1, 9, 4, 7, 3]]'

LLM with function calling will produce json input to pre-specified function ("create_matrix()" in function call)

In [21]:
llm_result_fc = llm.invoke("can you create a random 3x5 matrix?", 
                        tools = [convert_to_openai_tool(create_matrix)])

llm_result_fc

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hSJ0FnN67g7Itf0A3ZkLfyb8', 'function': {'arguments': '{"r":3,"c":5}', 'name': 'create_matrix'}, 'type': 'function'}]})

Notice that content will be null, but this will be report json input ready to be passed to actual function

In [23]:
llm_result_fc.content

''

In [45]:
llm_result_fc.dict()["additional_kwargs"]["tool_calls"]

[{'id': 'call_hSJ0FnN67g7Itf0A3ZkLfyb8',
  'function': {'arguments': '{"r":3,"c":5}', 'name': 'create_matrix'},
  'type': 'function'}]